In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# Read the data into a pandas DataFrame.
# This data is all Olympics data from 1980, up to 2018 as pulled from the original sheet.
df = pd.read_csv('120_Years_Test_Data/120_year_test_data.csv')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


In [5]:
#Create a dataframe with only the Summer Olympics data.
Summer_Olympics_df = df[df['Season'] == 'Summer']
Summer_Olympics_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
26,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN,Netherlands,NaN


In [6]:
#Create a dataframe with only the Medal Winners from the Summer Olympics.
Medal_Winner_df = Summer_Olympics_df[df['Medal'].notnull()].reset_index(drop=True)
Medal_Winner_df.head()

C:\Users\Topher\AppData\Local\Temp/ipykernel_32240/1786840130.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Medal_Winner_df = Summer_Olympics_df[df['Medal'].notnull()].reset_index(drop=True)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
1,15,Arvo Ossian Aaltonen,M,30.0,NaN,NaN,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze,Finland,NaN
2,15,Arvo Ossian Aaltonen,M,30.0,NaN,NaN,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze,Finland,NaN
3,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze,Finland,NaN
4,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold,Finland,NaN


In [7]:
Medal_Winner_df.shape

(34088, 17)

#### SVM and K_Nearest Neighbors to predict the number of medals won by a country.

In [14]:
#K Nearest Neighbors Data Analysis
Summer_Olympics_df['Medal'] = Summer_Olympics_df['Medal'].fillna('None')
#remove all none medals
Summer_Olympics_df1 = Summer_Olympics_df[Summer_Olympics_df['Medal'] != 'None']
#Create X and Y variables for predictions
ohe = np.array(Summer_Olympics_df1.drop(['ID', 'Name', 'Season', 'Medal', 'notes', 'NOC', 'Games', 'Event', 'region', 'Age', 'Height', 'Weight','Year','City'],1))
#v_data =  np.array(Summer_Olympics_df.drop(['ID', 'Name', 'Season', 'Medal', 'notes', 'NOC', 'Games', 'Event', 'region','Team','Sex','City', 'Sport'],1))
y = np.array(Summer_Olympics_df1['Medal'])

#Create transform for making all data numeric instead of categorical
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
ohe1 = enc.fit_transform(ohe).toarray()

#X = np.concatenate([ohe, v_data], axis=1)
X = ohe1

X.shape

C:\Users\Topher\AppData\Local\Temp/ipykernel_32240/1923736557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Summer_Olympics_df['Medal'] = Summer_Olympics_df['Medal'].fillna('None')
C:\Users\Topher\AppData\Local\Temp/ipykernel_32240/1923736557.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ohe = np.array(Summer_Olympics_df1.drop(['ID', 'Name', 'Season', 'Medal', 'notes', 'NOC', 'Games', 'Event', 'region', 'Age', 'Height', 'Weight','Year','City'],1))


(34088, 534)

In [15]:
X = X.astype(int)
X[np.isnan(X)] = 0

In [16]:
#Split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [17]:
#fit data to kneighbors model
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)

KNeighborsClassifier()

In [18]:
#Show us a score of the predictability of this model
clf.score(X_test, y_test)

0.4882174635768065

In [34]:
from sklearn.neural_network import MLPClassifier
clf_nn = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 100, max_iter =3000).fit(X_train, y_train)

In [35]:
clf_nn.score(X_test, y_test)

0.5244939865063069

In [28]:
test_val = enc.fit(ohe).transform(np.array(['M','United States','Athletics']).reshape(1,-1)).toarray()

In [29]:
clf_nn.predict(test_val)

array(['Gold'], dtype='<U6')

In [30]:
clf_nn.predict_proba(test_val)

array([[0.20000334, 0.49078248, 0.30921418]])

In [31]:
clf_nn.classes_

array(['Bronze', 'Gold', 'Silver'], dtype='<U6')

In [32]:
clf.predict_proba(test_val)

array([[0.2, 0.4, 0.4]])

B   #   #   Sex
    #
G   #   #   City
    #
S   #       Sport

.2x1 + .3x2 + .5x3...

3 4 2 4